# Examples of Assistory Library: Optimization

Table of content
- Static Production
- Rapid Production

In [1]:
import sys, os
os.chdir('..')

assert 'assistory' in os.listdir(os.getcwd())
sys.path.append(os.getcwd())

from assistory import game

## Static Production
See [this readme](./../docs/optimal_production.md) for concept behind the static production optimization.

In the following an example is shown


In [2]:
from assistory.optim.static_production_problem_config import StaticProductionLPConfig

problem_config = StaticProductionLPConfig(
    sell_rate_lower_limits=game.ItemValues({
        'Desc_IronScrew_C': 1000
    }), # goal is to produce 1000 screws per minute
    available_resource_nodes=game.ResourceNodeValues({
        game.get_resource_node_name('Desc_Coal_C', fracking=False): 5,
        game.get_resource_node_name('Desc_OreCopper_C', fracking=False): 5,
        game.get_resource_node_name('Desc_OreIron_C', fracking=False): 5,
    }), # these resource nodes are available
    unlocked_recipes=game.RecipeFlags(game.RECIPE_NAMES_AUTOMATED), # all recipes unlocked
    base_power=0, # power production must be included
    base_item_rate=game.ItemValues(), # No existing production
    minimize_resource_node_usage=True, # Objective
    weights_resource_node=game.ResourceNodeValues({
        game.get_resource_node_name('Desc_Coal_C', fracking=False): 0,
        game.get_resource_node_name('Desc_OreCopper_C', fracking=False): 1,
        game.get_resource_node_name('Desc_OreIron_C', fracking=False): 1,
    }), # coal can be used freely, but usage of copper and iron should be minimized
)

Now, the problem is set up. The next step is to run the optimization and to retrieve the result when possible.

In [3]:
from assistory.optim.static_production_problem import StaticProductionLP
from assistory.optim.static_flow_problem import ReturnCode

problem = StaticProductionLP(problem_config)

ret = problem.optimize()
if ret == ReturnCode.OPTIMAL:
    objective_val = round(problem.objective_value, 3)
    print('Found optimal solution:', objective_val, 'resource nodes used')

recipes_used = problem.get_recipes_used()
print('Resource nodes used:')
recipes_used.get_resource_nodes_used().round(3).pprint(ignore_value=0)

Found optimal solution: 0.093 resource nodes used
Resource nodes used:
Desc_Coal_C-non_fracking    0.668
Desc_OreIron_C-non_fracking 0.093


The production plan now can be retrieved.

In [4]:
print('Recipes used:')
recipes_used.round(3).pprint(ignore_value=0)

Recipes used:
Recipe_Alternate_IngotSteel_1_C  1.042
Recipe_Alternate_PureIronIngot_C 0.641
Recipe_Alternate_SteelRod_C      5.208
Recipe_GeneratorCoalCoal_C       2.564
Recipe_MinerMk2Coal_C            0.668
Recipe_MinerMk3OreIron_C         0.093
Recipe_Screw_C                   25.0
Recipe_WaterPumpWater_C          1.068


## Rapid Production
See [this readme](./../rapid_production.md) for concept behind the rapid production optimization.

In the following an example of using the `IterativeProductionProblem` class is shown. It minimizes the recipes used for a fixes number of steps and step duration.


In [5]:
from assistory.optim.rapid_production_problem_config import RapidProductionProblemConfig

rapid_problem_config = RapidProductionProblemConfig(
    S=game.ItemValues({
        'Desc_OreIron_C': 10,
        'Desc_IronRod_C': 5,
        'Desc_Wire_C': 8
    }),
    G=game.ItemValues({
        'Desc_IronIngot_C': 100
    }),
    unlocked_recipes=game.RecipeFlags({
        'Recipe_HandcraftOreIron_C',
        'Recipe_IngotIron_C',
        'Recipe_IronPlate_C',
        'Recipe_IronRod_C',
    }),
    maximal_step_count=5, # maximal number of steps
    step_duration=1,
)

The solution is a rapid plan. It's steps can be accessed via the attibutes or they can be printed conventiently by the function `print_debug` of the plan.

In [6]:
from assistory.optim.iterative_production_problem import IterativeProductionProblem

iterative_problem_config = rapid_problem_config.get_iterative_production_problem_config(5)
problem = IterativeProductionProblem(iterative_problem_config)
status = problem.optimize()
assert status == ReturnCode.OPTIMAL
recipe_count = problem.objective_value
print('Recipes used:', recipe_count)

rapid_plan = problem.get_rapid_plan().round(3)
print('Recipes used by step (Step, Handcraft, Automated):')
for i in range(5):
    print(
        i,
        rapid_plan.steps_recipes_handcraft[i],
        rapid_plan.steps_recipes_automated[i]
    )

Recipes used: 2.375
Recipes used by step (Step, Handcraft, Automated):
0 {'Recipe_IngotIron_C': 0.125} {}
1 {} {}
2 {'Recipe_HandcraftOreIron_C': 0.75} {}
3 {'Recipe_HandcraftOreIron_C': 0.375, 'Recipe_IngotIron_C': 0.375} {}
4 {'Recipe_IngotIron_C': 0.75} {}


When, also the number of steps should be minized, use the following code

In [7]:
from assistory.optim.rapid_production_problem import RapidProductionProblem

problem = RapidProductionProblem(rapid_problem_config)
status = problem.optimize()
assert status == ReturnCode.OPTIMAL
step_count = problem.objective_value
print('Steps used:', step_count)

rapid_plan = problem.get_rapid_plan()
recipes_used_cnt = sum(
    sum(rapid_plan.steps_recipes_automated[step_id].values())
    + sum(rapid_plan.steps_recipes_handcraft[step_id].values())
    for step_id in range(step_count)
)
print('Total recipes used:', round(recipes_used_cnt, 3))

rapid_plan = rapid_plan.round(3)
print('Recipes used by step (handcraft, automated):')
for i in range(step_count):
    print(
        i,
        rapid_plan.steps_recipes_handcraft[i],
        rapid_plan.steps_recipes_automated[i]
    )

Steps used: 3
Total recipes used: 3.0
Recipes used by step (handcraft, automated):
0 {'Recipe_HandcraftOreIron_C': 0.75} {'Recipe_IngotIron_C': 1.0}
1 {'Recipe_HandcraftOreIron_C': 0.375, 'Recipe_IngotIron_C': 0.375} {}
2 {'Recipe_IngotIron_C': 0.5} {}


As can be observed, now only 3 steps are required. However, the number of recipes used is slightly larger compared to when using 5 steps.